In [ ]:
import cv2
import tensorflow as tf
from flask import Flask, render_template_string, request, jsonify
from tensorflow.keras.preprocessing import image
import numpy as np
import io
import base64

app = Flask(__name__)

# Loading the training model
model = tf.keras.models.load_model('Skin Oiliness Severity Detection Final Inesha 12-30.h5')

# Load the pre-trained Haar Cascade face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')


def preprocess_img(img):
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = tf.keras.applications.mobilenet.preprocess_input(img)
    return img


def predict_oiliness(img):
    img = preprocess_img(img)
    prediction = model.predict(img)
    return prediction


def detect_faces(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    return faces


In [ ]:
@app.route('/')
def index():
    html_content = """
      <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Skin Oiliness Detection</title>
        <style>
            body {
                font-family: Arial, sans-serif;
                background-color: #f7f7f7; /* Light gray background */
                margin: 0;
                padding: 0;
  background-image: url('https://www.skinician.com/cdn/shop/articles/Oily_Skincare_Routine_and_Tips_for_Women_in_Their_40s_25d18970-9f45-4f75-a23d-f1b01e03c568_1100x.jpg?v=1663940214');
    background-size: cover;
    background-repeat: no-repeat;
    display: flex;
    justify-content: center;
    align-items: center;
    height: 100vh;

            }
            .container {
                max-width: 1500px;
                margin: 50px ;
                padding: 20px;
                background-color: #fff; /* White container background */
                border-radius: 40px;
                box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
            }
            h1 {
                text-align: center;
                color: #333;
            }
            form {
                text-align: center;
                margin-top: 20px;
            }
            label {
                display: block;
                margin-bottom: 10px;
                font-weight: bold;
            }
            input[type="radio"], input[type="file"], button {
                  display: block;
    margin-top: 10px;
    padding: 10px 20px;
    border: none;
    background-color: #28aedf;
    color: #fff;
    text-transform: uppercase;
    cursor: pointer;
    border-radius: 100px;
            }
            button {
                padding: 10px 20px;
                background-color: #007bff; /* Blue button background */
                color: #fff;
                border: none;
                border-radius: 4px;
                cursor: pointer;
                transition: background-color 0.3s;
            }
            button:hover {
                background-color: #0056b3; /* Darker blue on hover */
            }
            #webcam_div {
                display: none;
            }
            video {
                width: 100%;
                height: auto;
                margin-bottom: 10px;
            }
        </style>
    </head>
    <body>
        <div class="container">
<img class="logo" src="https://png.pngtree.com/png-vector/20221017/ourmid/pngtree-beauty-women-face-silhouette-spa-makeup-vector-vector-png-image_39729534.png" alt="Logo" width="100px" height="100px" class="center">
            <h1>Skin Oiliness Detection</h1>
            <form id="upload_form" action="/upload" method="post" enctype="multipart/form-data">
                <label for="device_button">
                    <input type="radio" name="source" id="device_button" value="device" onclick="toggleWebcam()" checked> Upload from device
                </label><br>
                <label for="webcam_button">
                    <input type="radio" name="source" id="webcam_button" value="webcam" onclick="toggleWebcam()"> Capture from webcam
                </label><br>
                <div id="webcam_div">
                    <video id="video" width="640" height="480" autoplay></video><br>
                    <button onclick="captureImage()">Capture Image from Webcam</button><br>
                    <input type="hidden" name="image_data" id="image_data">
                </div>
                <input type="file" name="file" id="file_input">
                <button type="submit">Upload</button>
            </form>
        </div>

        <script>
            function toggleWebcam() {
                var webcamDiv = document.getElementById('webcam_div');
                var fileInput = document.getElementById('file_input');
                var webcamButton = document.getElementById('webcam_button');
                if (webcamButton.checked) {
                    webcamDiv.style.display = 'block';
                    fileInput.style.display = 'none';
                    startWebcam();
                } else {
                    webcamDiv.style.display = 'none';
                    fileInput.style.display = 'block';
                }
            }

            function startWebcam() {
                navigator.mediaDevices.getUserMedia({ video: true })
                    .then(function(stream) {
                        var video = document.getElementById('video');
                        video.srcObject = stream;
                    })
                    .catch(function(err) {
                        console.log("An error occurred: " + err);
                    });
            }

            function captureImage() {
                var video = document.getElementById('video');
                var canvas = document.createElement('canvas');
                canvas.width = video.videoWidth;
                canvas.height = video.videoHeight;
                var context = canvas.getContext('2d');
                context.drawImage(video, 0, 0, canvas.width, canvas.height);
                var imgData = canvas.toDataURL('image/png');
                document.getElementById('image_data').value = imgData;
            }
        </script>
    </body>
    </html>


    """
    return render_template_string(html_content)


In [ ]:
@app.route('/upload', methods=['POST'])
def upload():
    source = request.form.get('source')
    if source == 'device':
        file = request.files['file']
        if file.filename == '':
            return jsonify({'error': 'No selected file'})
        img = image.load_img(io.BytesIO(file.read()), target_size=(224, 224))
    else:
        image_data = request.form['image_data']
        # Decode base64 image data
        img_bytes = io.BytesIO(base64.b64decode(image_data.split(',')[1]))
        # Convert the image bytes to a PIL image
        img = image.load_img(img_bytes, target_size=(224, 224))

    if source == 'webcam':
        # Convert the base64 image to OpenCV format
        encoded_data = image_data.split(',')[1]
        nparr = np.frombuffer(base64.b64decode(encoded_data), np.uint8)
        webcam_img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

        # Detect faces in the webcam image
        faces = detect_faces(webcam_img)
        if len(faces) == 0:
            return jsonify({'error': 'No face detected'})

    # Predict skin oiliness level
    prediction = predict_oiliness(img)

    # Get the index of the class with the highest probability
    predicted_class_index = np.argmax(prediction)

    # Map the class index to the corresponding label
    labels = ["Oily Skin", "Normal Skin", "Dry Skin"]
    predicted_label = labels[predicted_class_index]

    return jsonify({'prediction': predicted_label})

if __name__ == '__main__':
    app.run(port=5001)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
